# Ahmed's API Key

In [ ]:
SOLD_URL_ENDPOINT = 'https://redfin-com-data.p.rapidapi.com/properties/search-sold'
AUTOCOMPLETE_URL_ENDPOINT = "https://redfin-com-data.p.rapidapi.com/properties/auto-complete"
RAPIDAPI_KEY = 'e30df9c98fmshc8107a744002a40p1825e4jsn566f389d12de'

# Marwa's API Key

In [ ]:

headers = {
	"x-rapidapi-key": "d189a8f130msh1de215a13d1f7b0p101758jsnfb514e240299",
	"x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
}


# Salama API Key

In [ ]:
headers = {
	"x-rapidapi-key": "c8428f1837msh24e02ec82ded1efp1692e8jsn591739e697c4",
	"x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
}

In [ ]:
import requests
import pandas as pd
import logging
import datetime

# Constants and Configurations
RAPIDAPI_KEY = 'e30df9c98fmshc8107a744002a40p1825e4jsn566f389d12de'
API_URL = 'https://redfin-com-data.p.rapidapi.com/properties/search-sold'
CSV_FILE_PATH = 'Data/Export-NH-counties (Cleaned).xlsx - Sheet1.csv'
EXCEL_FOLDER = 'Zillow'
EXCEL_FILE_PREFIX = 'Letters'

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

def fetch_data_from_redfin(region_id, lotSize, hoaFees, soldwithin, homeType, limit):
    url = "https://redfin-com-data.p.rapidapi.com/properties/search-sold"

    querystring = {
        "regionId": region_id,
        "lotSize": lotSize,
        "hoaFees": hoaFees,
        "soldWithin": soldwithin,
        "homeType": homeType,
        "limit": limit
    }

    headers = {
	"x-rapidapi-key": "c8428f1837msh24e02ec82ded1efp1692e8jsn591739e697c4",
	"x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
    }

    try:
        response = requests.get(url, headers=headers, params=querystring)
        response.raise_for_status()  # Raise an exception for bad status codes
        data = response.json().get('data', [])
        return data
    except requests.exceptions.RequestException as e:
        logging.error(f"Request failed: {e}")
        return []

def process_property_data(properties, region_id, zipcode):
    processed_data = []

    for property_data in properties:
        home_data = property_data.get("homeData", {})
        price_info = home_data.get("priceInfo", {})
        lot_size_info = home_data.get("lotSize", {})

        price = price_info.get("amount", "Not available")
        lot_size_sqft = lot_size_info.get("amount", "Not available")

        # Convert lot size from sqft to acres if available
        if lot_size_sqft != "Not available":
            try:
                lot_size_acres = float(lot_size_sqft) / 43560.0  # converting from sqft to acres
                lot_size_acres = round(lot_size_acres, 2)  # Round to 2 decimal places
                lot_size_str = lot_size_acres
            except ValueError:
                lot_size_str = "Not available"
        else:
            lot_size_str = "Not available"

        processed_data.append({
            'Price': price,
            'Zipcode': zipcode,
            'Region ID': region_id,
            'Lot Size': lot_size_str
        })

    return pd.DataFrame(processed_data)

def save_data_to_excel(df):
    current_month = datetime.datetime.now().strftime("%B")
    current_year = datetime.date.today().year
    excel_file = f'{EXCEL_FOLDER}/{EXCEL_FILE_PREFIX}_{current_month}_{current_year}.xlsx'
    df.to_excel(excel_file, index=False)
    logging.info(f"All data saved to {excel_file}")

def get_region_id(zipcode):
    url = "https://redfin-com-data.p.rapidapi.com/properties/auto-complete"

    querystring = {"query": zipcode}

    headers = {
	"x-rapidapi-key": "c8428f1837msh24e02ec82ded1efp1692e8jsn591739e697c4",
	"x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)

    data = response.json()

    # Access the id value
    id_value = data['data'][0]['rows'][0]['id']
    return id_value

def load_zipcodes_from_csv(csv_file):
    df = pd.read_csv(csv_file, dtype={'ZIP': str})
    df['ZIP'] = "0" + df['ZIP']
    zipcodes_float_list = df['ZIP'].dropna().unique()
    return zipcodes_float_list

def main():
    zipcodes = load_zipcodes_from_csv(CSV_FILE_PATH)
    lotSize = "43560,174240"
    soldwithin = 365
    homeType = 5
    hoaFees = 0
    limit = 20
    processed_data = pd.DataFrame()
    for zipcode in zipcodes:
        region_id = get_region_id(zipcode)

        properties = fetch_data_from_redfin(region_id, lotSize, hoaFees, soldwithin, homeType, limit)

        if properties:
            processed_data = pd.concat([process_property_data(properties, region_id, zipcode), processed_data], axis=0)

            save_data_to_excel(processed_data)
        else:
            logging.warning(f"No data fetched for zipcode {zipcode}.")

if __name__ == "__main__":
    main()


In [ ]:
import pandas as pd

# Load the first dataset
df1 = pd.read_csv('Data/Export-CarrollCnty-NH-zips (CLeaned).xlsx - Sheet1.csv', dtype={'ZIP': str})

# Load the second dataset containing unique zip codes and average prices
df2 = pd.read_csv('Cleaned Zipcodes.csv', dtype={'ZIP': str})

# Ensure the ZIP columns are of the same type (string is often safest for ZIP codes)
df1['ZIP'] = df1['ZIP'].astype(str)
df2['ZIP'] = df2['ZIP'].astype(str)

# Merge the datasets on the ZIP code
merged_df = pd.merge(df1, df2, on='ZIP', how='left')

# Save the merged DataFrame to a new CSV file
merged_df.to_csv('path_to_output_dataset.csv', index=False)

print("Merged dataset saved successfully.")
